In [2]:
import pymongo
from pymongo import MongoClient
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing import sequence
import urllib.parse

In [3]:
import time

In [38]:
from keras.utils import np_utils
# Import the Keras libraries and packages
%env KERAS_BACKEND=tensorflow
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Input,Embedding
from keras.layers import Dropout
from keras.optimizers import SGD

env: KERAS_BACKEND=tensorflow


### 寫一個模型
輸入：歷史交易資料，以周為單為<br/>
輸出：這個交易者正不正常<br/>
正常資料：輸出標記為0，定義為交易額度非常均勻，無極端值<br/>
不正常資料：輸出標記為1，定義為某次交易的額度遠超過其他額度，有極端值<br/><br/>
發現問題描述：如果在第一層新增一層BatchNormalization，則可順利偵測並獲得極高準確率<br/>否則會失敗


使用模型：LSTM<br/>
輸入參數使用數字大小，然後數字大小是交易額


In [119]:
np.random.randint(3,37,(3,3))

array([[17, 11, 15],
       [18, 26, 10],
       [ 8, 30, 32]])

In [155]:
data = []
answer = []
for i in range(1000):
    temp = []
    origin = np.random.randint(10,200000)
    for d in range(100):
        origin += np.random.randint(-10,10)
        if origin<=0:
            origin = 0
        temp.append(origin)
    answer.append(0)
    data.append(temp)

In [127]:
data = np.array(data) #正常資料

In [137]:
if False==bool(90%30):
    print(87)

87


In [269]:
bad_data = []
bad_answer = []
for i in range(1000):
    temp = []
    origin = np.random.randint(1,2000)
    trigger = 0
    for d in range(100):
        origin += np.random.randint(-10,10)
        if np.random.randint(-20,1)>-2:
            #print(i)
            temp2 = origin
            origin+=np.random.randint(1,9)*origin
            temp.append(origin)
            origin = temp2
            trigger = 1
        else:temp.append(origin)
    bad_data.append(temp)
    bad_answer.append(trigger)

In [270]:
np.array(bad_data).shape

(1000, 100)

In [272]:
0 in bad_answer

False

In [273]:
raw_data = np.array(list(data) + list(bad_data))

In [274]:
raw_answer = np.array(answer+bad_answer)

#### data完成 準備製作模型

In [275]:
raw_answer.shape

(2000,)

In [276]:
raw_data.shape

(2000, 100)

In [277]:
import sklearn

In [278]:
import sklearn.model_selection

In [279]:
train_x, test_x, train_y, test_y = sklearn.model_selection.train_test_split(raw_data, raw_answer, test_size = 0.33, random_state= 87)

In [300]:
from keras.layers import pooling, Flatten, BatchNormalization


In [340]:
model = Sequential()
k=80
model.add(BatchNormalization(input_shape=(100,)))

#model.add(Dense(80,  activation = 'relu',input_shape=(100,)))
model.add(Dense(80,  activation = 'relu'))

model.add(Dense(40,activation='relu'))
#model.add(Dropout(0.4))
model.add(Dense(20,activation='relu'))

#model.add(Flatten())
#model.add(Dropout(0.4))
model.add(Dense(1, activation = "sigmoid"))

In [341]:
model.compile(loss='mse',
              optimizer = "adam", #本來是ADAM
              metrics = ['accuracy'])

In [342]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_6 (Batch (None, 100)               400       
_________________________________________________________________
dense_100 (Dense)            (None, 80)                8080      
_________________________________________________________________
dense_101 (Dense)            (None, 40)                3240      
_________________________________________________________________
dense_102 (Dense)            (None, 20)                820       
_________________________________________________________________
dense_103 (Dense)            (None, 1)                 21        
Total params: 12,561
Trainable params: 12,361
Non-trainable params: 200
_________________________________________________________________


In [343]:
model.fit(np.array(train_x), np.array(train_y), batch_size=100, epochs= 10)

Epoch 1/10
1340/1340 [==============================] - 1s 540us/step - loss: 0.1731 - acc: 0.7530
Epoch 2/10
1340/1340 [==============================] - 0s 22us/step - loss: 0.0925 - acc: 0.8679
Epoch 3/10
1340/1340 [==============================] - 0s 22us/step - loss: 0.0810 - acc: 0.8806
Epoch 4/10
1340/1340 [==============================] - 0s 21us/step - loss: 0.0774 - acc: 0.8948
Epoch 5/10
1340/1340 [==============================] - 0s 22us/step - loss: 0.0712 - acc: 0.9022
Epoch 6/10
1340/1340 [==============================] - 0s 18us/step - loss: 0.0665 - acc: 0.9149
Epoch 7/10
1340/1340 [==============================] - 0s 21us/step - loss: 0.0609 - acc: 0.9231
Epoch 8/10
1340/1340 [==============================] - 0s 23us/step - loss: 0.0547 - acc: 0.9321
Epoch 9/10
1340/1340 [==============================] - 0s 22us/step - loss: 0.0495 - acc: 0.9418
Epoch 10/10
1340/1340 [==============================] - 0s 23us/step - loss: 0.0496 - acc: 0.9463


In [349]:
model.evaluate(test_x, test_y)

660/660 [==============================] - 0s 22us/step


[0.03758163635026325, 0.9499999992775194]

In [345]:
model2 = Sequential()
#model2.add(BatchNormalization(input_shape=(100,)))

model2.add(Dense(80,  activation = 'relu',input_shape=(100,)))
#model.add(Dense(80,  activation = 'relu'))

model2.add(Dense(40,activation='relu'))
#model.add(Dropout(0.4))
model2.add(Dense(20,activation='relu'))

#model.add(Flatten())
#model.add(Dropout(0.4))
model2.add(Dense(1, activation = "sigmoid"))

In [346]:
model2.compile(loss='mse',
              optimizer = "adam", #本來是ADAM
              metrics = ['accuracy'])

In [347]:
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_104 (Dense)            (None, 80)                8080      
_________________________________________________________________
dense_105 (Dense)            (None, 40)                3240      
_________________________________________________________________
dense_106 (Dense)            (None, 20)                820       
_________________________________________________________________
dense_107 (Dense)            (None, 1)                 21        
Total params: 12,161
Trainable params: 12,161
Non-trainable params: 0
_________________________________________________________________


In [348]:
model2.fit(np.array(train_x), np.array(train_y), batch_size=100, epochs= 10)

Epoch 1/10
1340/1340 [==============================] - 1s 509us/step - loss: 0.4729 - acc: 0.5261
Epoch 2/10
1340/1340 [==============================] - 0s 19us/step - loss: 0.5286 - acc: 0.4709
Epoch 3/10
1340/1340 [==============================] - 0s 20us/step - loss: 0.5016 - acc: 0.4985
Epoch 4/10
1340/1340 [==============================] - 0s 19us/step - loss: 0.5007 - acc: 0.4993
Epoch 5/10
1340/1340 [==============================] - 0s 19us/step - loss: 0.5007 - acc: 0.4993
Epoch 6/10
1340/1340 [==============================] - 0s 18us/step - loss: 0.5007 - acc: 0.4993
Epoch 7/10
1340/1340 [==============================] - 0s 19us/step - loss: 0.5007 - acc: 0.4993
Epoch 8/10
1340/1340 [==============================] - 0s 20us/step - loss: 0.5007 - acc: 0.4993
Epoch 9/10
1340/1340 [==============================] - 0s 16us/step - loss: 0.5007 - acc: 0.4993
Epoch 10/10
1340/1340 [==============================] - 0s 19us/step - loss: 0.5007 - acc: 0.4993


In [350]:
model2.evaluate(test_x, test_y)

660/660 [==============================] - 0s 389us/step


[0.49848484920732905, 0.5015151516957717]

In [371]:
model3 = Sequential()
model3.add(BatchNormalization(input_shape=(100,)))

In [372]:
model3.compile(loss='mse',
              optimizer = "adam", #本來是ADAM
              metrics = ['accuracy'])

In [373]:
model3.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_9 (Batch (None, 100)               400       
Total params: 400
Trainable params: 200
Non-trainable params: 200
_________________________________________________________________


In [376]:
a = np.zeros(100)

a+= np.random.randint(1,10)

a[8] += 1000

a.shape

a

array([   9.,    9.,    9.,    9.,    9.,    9.,    9.,    9., 1009.,
          9.,    9.,    9.,    9.,    9.,    9.,    9.,    9.,    9.,
          9.,    9.,    9.,    9.,    9.,    9.,    9.,    9.,    9.,
          9.,    9.,    9.,    9.,    9.,    9.,    9.,    9.,    9.,
          9.,    9.,    9.,    9.,    9.,    9.,    9.,    9.,    9.,
          9.,    9.,    9.,    9.,    9.,    9.,    9.,    9.,    9.,
          9.,    9.,    9.,    9.,    9.,    9.,    9.,    9.,    9.,
          9.,    9.,    9.,    9.,    9.,    9.,    9.,    9.,    9.,
          9.,    9.,    9.,    9.,    9.,    9.,    9.,    9.,    9.,
          9.,    9.,    9.,    9.,    9.,    9.,    9.,    9.,    9.,
          9.,    9.,    9.,    9.,    9.,    9.,    9.,    9.,    9.,
          9.])

In [377]:
model3.predict(np.array(a).reshape(1,-1))

array([[   8.995503,    8.995503,    8.995503,    8.995503,    8.995503,
           8.995503,    8.995503,    8.995503, 1008.49585 ,    8.995503,
           8.995503,    8.995503,    8.995503,    8.995503,    8.995503,
           8.995503,    8.995503,    8.995503,    8.995503,    8.995503,
           8.995503,    8.995503,    8.995503,    8.995503,    8.995503,
           8.995503,    8.995503,    8.995503,    8.995503,    8.995503,
           8.995503,    8.995503,    8.995503,    8.995503,    8.995503,
           8.995503,    8.995503,    8.995503,    8.995503,    8.995503,
           8.995503,    8.995503,    8.995503,    8.995503,    8.995503,
           8.995503,    8.995503,    8.995503,    8.995503,    8.995503,
           8.995503,    8.995503,    8.995503,    8.995503,    8.995503,
           8.995503,    8.995503,    8.995503,    8.995503,    8.995503,
           8.995503,    8.995503,    8.995503,    8.995503,    8.995503,
           8.995503,    8.995503,    8.995503,    8

In [378]:
model3.fit(a.reshape(1,-1), np.zeros(100).reshape(1,-1))

Epoch 1/1
1/1 [==============================] - 1s 680ms/step - loss: 0.0000e+00 - acc: 1.0000


In [379]:
model3.predict(np.array(a).reshape(1,-1))

array([[-0.00027466, -0.00027466, -0.00027466, -0.00027466, -0.00027466,
        -0.00027466, -0.00027466, -0.00027466, -0.03125   , -0.00027466,
        -0.00027466, -0.00027466, -0.00027466, -0.00027466, -0.00027466,
        -0.00027466, -0.00027466, -0.00027466, -0.00027466, -0.00027466,
        -0.00027466, -0.00027466, -0.00027466, -0.00027466, -0.00027466,
        -0.00027466, -0.00027466, -0.00027466, -0.00027466, -0.00027466,
        -0.00027466, -0.00027466, -0.00027466, -0.00027466, -0.00027466,
        -0.00027466, -0.00027466, -0.00027466, -0.00027466, -0.00027466,
        -0.00027466, -0.00027466, -0.00027466, -0.00027466, -0.00027466,
        -0.00027466, -0.00027466, -0.00027466, -0.00027466, -0.00027466,
        -0.00027466, -0.00027466, -0.00027466, -0.00027466, -0.00027466,
        -0.00027466, -0.00027466, -0.00027466, -0.00027466, -0.00027466,
        -0.00027466, -0.00027466, -0.00027466, -0.00027466, -0.00027466,
        -0.00027466, -0.00027466, -0.00027466, -0.0